# Submission for Kryptonite-N

# Decision Tree

In [2]:
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from codecarbon import track_emissions

def preprocess_data(n, use_split = True):
    Xs = np.load(f"/home/moritz/maths-for-ml/Kryptonite-N/Datasets/kryptonite-{n}-X.npy")
    Ys = np.load(f"/home/moritz/maths-for-ml/Kryptonite-N/Datasets/kryptonite-{n}-y.npy")
    df_x = pd.DataFrame(Xs)
    df_y = pd.Series(Ys)  
    df_x = (df_x >= 0.5).astype(int)
    return train_test_split(df_x, df_y, test_size=0.2) if use_split else df_x,df_y

def preprocess_prediction_data(n):
    Xs = np.load(f"/home/moritz/maths-for-ml/Kryptonite-N/Datasets/hidden-kryptonite-{n}-X.npy")
    df_x = pd.DataFrame(Xs)
    df_x = (df_x >= 0.5).astype(int)
    return df_x
    

In [9]:

from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

@track_emissions()
def fit_decision_tree(X_train,y_train):
    decision_tree = DecisionTreeClassifier(max_depth=15,min_samples_leaf=2,min_samples_split=2)
    decision_tree.fit(X_train, y_train)
    return decision_tree
   
    
@track_emissions
def predict_decision_tree(decision_tree, X_test):
    return decision_tree.predict(X_test)
    




In [ ]:
for n in [9,12,15,18]:
    X_train, X_test, y_train, y_test = preprocess_data(n)
    decision_tree = fit_decision_tree(X_train, y_train)
    y_pred = predict_decision_tree(decision_tree, X_test)
    accuracy = accuracy_score(y_test, y_pred)
    print(f"Decision Tree Accuracy: {accuracy:.4f} for n = {n}")

## Neural Network

In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LeakyReLU, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

@track_emissions()
def fit_neural_network(X_train,y_train, use_early_stopping = True):
    model = Sequential([
        Input(shape=(X_train.shape[1],), sparse=True),
        Dense(128),
        LeakyReLU(alpha=0.1),
        
        Dense(64),
        LeakyReLU(alpha=0.1),

        Dense(32),
        LeakyReLU(alpha=0.1),
        
        Dense(1, activation='sigmoid')
    ])

    
    model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])
    

    early_stopping = EarlyStopping(
        monitor='val_loss', 
        patience=20, 
        restore_best_weights=True
    )

    
    model.fit(
        X_train, y_train,
        epochs= 500,
        batch_size=32,
        validation_split=0.1, 
        verbose=1,
        callbacks = [early_stopping] if use_early_stopping else []
    )
    return model
@track_emissions
def predict_neural_network(model, X_test):
    return (model.predict(X_test) > 0.5).astype(int)


In [ ]:

for n in [9,12,15,18]:
    X_train, X_test, y_train, y_test = preprocess_data(n)
    decision_tree = fit_neural_network(X_train, y_train)
    y_pred = predict_neural_network(decision_tree, X_test)
    accuracy = accuracy_score(y_test, y_pred)
    print(f"Neural Netork Accuracy: {accuracy:.4f} for n = {n}")


# Prediction of the labels

## Neural Network

In [13]:
for n in [9,12,15,18]:
    print(f"{n} --------------------------------------------------------- ")
    X_train, y_train = preprocess_data(n,use_split=False)
    neural_network = fit_neural_network(X_train, y_train)
    X_test = preprocess_prediction_data(n)
    y_pred = predict_neural_network(neural_network, X_test)
    np.save(f"/home/moritz/maths-for-ml/Kryptonite-N/submission/y_predicted_{n}.npy", y_pred)
    
    


[codecarbon INFO @ 18:31:24] [setup] RAM Tracking...
[codecarbon INFO @ 18:31:24] [setup] GPU Tracking...
[codecarbon INFO @ 18:31:24] Tracking Nvidia GPU via pynvml


9 --------------------------------------------------------- 


[codecarbon INFO @ 18:31:25] [setup] CPU Tracking...
[codecarbon WARNING @ 18:31:25] No CPU tracking mode found. Falling back on CPU constant mode. 
 Windows OS detected: Please install Intel Power Gadget to measure CPU

[codecarbon WARNING @ 18:31:27] We saw that you have a 12th Gen Intel(R) Core(TM) i9-12900H but we don't know it. Please contact us.
[codecarbon INFO @ 18:31:27] CPU Model on constant consumption mode: 12th Gen Intel(R) Core(TM) i9-12900H
[codecarbon INFO @ 18:31:27] >>> Tracker's metadata:
[codecarbon INFO @ 18:31:27]   Platform system: Windows-11-10.0.22631-SP0
[codecarbon INFO @ 18:31:27]   Python version: 3.12.7
[codecarbon INFO @ 18:31:27]   CodeCarbon version: 2.7.4
[codecarbon INFO @ 18:31:27]   Available RAM : 31.694 GB
[codecarbon INFO @ 18:31:27]   CPU count: 20
[codecarbon INFO @ 18:31:27]   CPU model: 12th Gen Intel(R) Core(TM) i9-12900H
[codecarbon INFO @ 18:31:27]   GPU count: 1
[codecarbon INFO @ 18:31:27]   GPU model: 1 x NVIDIA GeForce RTX 4060 Laptop 

Epoch 1/500
507/507 ━━━━━━━━━━━━━━━━━━━━ 1s 881us/step - accuracy: 0.5697 - loss: 0.6692 - val_accuracy: 0.9517 - val_loss: 0.2310
Epoch 2/500
507/507 ━━━━━━━━━━━━━━━━━━━━ 0s 748us/step - accuracy: 0.9478 - loss: 0.2246 - val_accuracy: 0.9650 - val_loss: 0.1604
Epoch 3/500
507/507 ━━━━━━━━━━━━━━━━━━━━ 0s 702us/step - accuracy: 0.9557 - loss: 0.1880 - val_accuracy: 0.9650 - val_loss: 0.1652
Epoch 4/500
507/507 ━━━━━━━━━━━━━━━━━━━━ 0s 761us/step - accuracy: 0.9561 - loss: 0.1886 - val_accuracy: 0.9650 - val_loss: 0.1561
Epoch 5/500
507/507 ━━━━━━━━━━━━━━━━━━━━ 0s 699us/step - accuracy: 0.9560 - loss: 0.1820 - val_accuracy: 0.9650 - val_loss: 0.1545
Epoch 6/500
507/507 ━━━━━━━━━━━━━━━━━━━━ 0s 709us/step - accuracy: 0.9575 - loss: 0.1773 - val_accuracy: 0.9650 - val_loss: 0.1572
Epoch 7/500
507/507 ━━━━━━━━━━━━━━━━━━━━ 0s 699us/step - accuracy: 0.9572 - loss: 0.1771 - val_accuracy: 0.9650 - val_loss: 0.1571
Epoch 8/500
507/507 ━━━━━━━━━━━━━━━━━━━━ 0s 688us/step - accuracy: 0.9570 - loss: 0

[codecarbon INFO @ 18:31:42] Energy consumed for RAM : 0.000050 kWh. RAM Power : 11.8851900100708 W


...

[codecarbon INFO @ 18:33:16] Energy consumed for all GPUs : 0.000023 kWh. Total GPU Power : 2.5823912005935177 W
[codecarbon INFO @ 18:33:16] Energy consumed for all CPUs : 0.000372 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 18:33:16] 0.000489 kWh of electricity used since the beginning.


1013/1013 ━━━━━━━━━━━━━━━━━━━━ 1s 724us/step - accuracy: 0.9664 - loss: 0.1124 - val_accuracy: 0.9636 - val_loss: 0.1761
Epoch 43/500
1013/1013 ━━━━━━━━━━━━━━━━━━━━ 1s 677us/step - accuracy: 0.9676 - loss: 0.1106 - val_accuracy: 0.9681 - val_loss: 0.1623
Epoch 44/500
1013/1013 ━━━━━━━━━━━━━━━━━━━━ 1s 678us/step - accuracy: 0.9672 - loss: 0.1100 - val_accuracy: 0.9644 - val_loss: 0.1693
Epoch 45/500
1013/1013 ━━━━━━━━━━━━━━━━━━━━ 1s 690us/step - accuracy: 0.9691 - loss: 0.1033 - val_accuracy: 0.9692 - val_loss: 0.1650
Epoch 46/500
1013/1013 ━━━━━━━━━━━━━━━━━━━━ 1s 699us/step - accuracy: 0.9687 - loss: 0.1035 - val_accuracy: 0.9672 - val_loss: 0.1698
Epoch 47/500
1013/1013 ━━━━━━━━━━━━━━━━━━━━ 1s 690us/step - accuracy: 0.9670 - loss: 0.1031 - val_accuracy: 0.9664 - val_loss: 0.1652
Epoch 48/500
1013/1013 ━━━━━━━━━━━━━━━━━━━━ 1s 683us/step - accuracy: 0.9690 - loss: 0.1029 - val_accuracy: 0.9644 - val_loss: 0.1819
Epoch 49/500
1013/1013 ━━━━━━━━━━━━━━━━━━━━ 1s 678us/step - accuracy: 0.971

[codecarbon INFO @ 18:33:24] 
Graceful stopping: collecting and writing information.
Please wait a few seconds...
[codecarbon INFO @ 18:33:24] Energy consumed for RAM : 0.000122 kWh. RAM Power : 11.8851900100708 W
[codecarbon INFO @ 18:33:24] Energy consumed for all GPUs : 0.000030 kWh. Total GPU Power : 3.0825660671405726 W
[codecarbon INFO @ 18:33:24] Energy consumed for all CPUs : 0.000475 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 18:33:24] 0.000628 kWh of electricity used since the beginning.
[codecarbon INFO @ 18:33:25] Done!

[codecarbon INFO @ 18:33:25] [setup] RAM Tracking...
[codecarbon INFO @ 18:33:25] [setup] GPU Tracking...
[codecarbon INFO @ 18:33:25] Tracking Nvidia GPU via pynvml
[codecarbon INFO @ 18:33:25] [setup] CPU Tracking...
[codecarbon WARNING @ 18:33:25] No CPU tracking mode found. Falling back on CPU constant mode. 
 Windows OS detected: Please install Intel Power Gadget to measure CPU

[codecarbon WARNING @ 18:33:26] We saw that you have a 12th Gen Inte

313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 618us/step


[codecarbon INFO @ 18:33:27] 
Graceful stopping: collecting and writing information.
Please wait a few seconds...
[codecarbon INFO @ 18:33:27] Energy consumed for RAM : 0.000001 kWh. RAM Power : 11.8851900100708 W
[codecarbon INFO @ 18:33:27] Energy consumed for all GPUs : 0.000002 kWh. Total GPU Power : 23.17515641359548 W
[codecarbon INFO @ 18:33:27] Energy consumed for all CPUs : 0.000009 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 18:33:27] 0.000012 kWh of electricity used since the beginning.
[codecarbon INFO @ 18:33:27] Done!

